In [ ]:
import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nnAudio import Spectrogram

In [ ]:
def extract_cqt_transformer(sample_rate, n_bins=84):
    """
    创建并返回CQT变换对象。
    
    CQT（Constant-Q Transform）是一种时频分析方法，具有对数频率分辨率，
    在低频处有更高的频率分辨率，在高频处有更高的时间分辨率。
    CQT特别适合音乐信号分析，因为它的频率分布与音乐音符的对数关系相匹配。
    
    Args:
        sample_rate (int): 音频采样率，决定了频谱分析的频率范围
        n_bins (int): CQT频率bins的数量，通常取84个（7个八度音程）
    
    Returns:
        Spectrogram.CQT: 初始化完成的CQT对象
        
    参数说明:
        - sr: 采样率
        - n_bins: CQT频率bins数量，决定输出特征维度
        - bins_per_octave: 每个八度音程的bins数量，通常为12
        - hop_length: 帧移，控制时间分辨率
        - fmin: 最低频率，通常设为C1音符频率
        - window: 窗函数类型，'hamming'提供良好的频率泄漏控制
        - center: 是否对信号进行中心填充
        - pad_mode: 填充模式，'reflect'进行镜像填充
        - trainable: 是否允许训练过程中调整参数
        - output_format: 输出格式，'Magnitude'表示幅度谱
        - verbose: 是否显示详细信息
    """
    # 初始化CQT变换对象
    cqt_transformer = Spectrogram.CQT(
        sr=sample_rate,
        n_bins=n_bins,
        bins_per_octave=12,
        hop_length=512,
        fmin=32.7,  # C1音符频率
        window='hamming',
        center=True,
        pad_mode='reflect',
        trainable=False,
        output_format='Magnitude',
        verbose=False
    )
    return cqt_transformer

In [ ]:
def save_cqt_to_csv(cqt_features, output_csv):
    """
    将CQT特征保存为CSV文件
    :param cqt_features: CQT特征矩阵
    :param output_csv: CSV文件路径
    """
    df = pd.DataFrame(cqt_features)
    df.to_csv(output_csv, index=False, header=False)
    print(f"CQT特征已保存至 {output_csv}")

In [ ]:
def plot_cqt_features(cqt_data, sr, output_image=None):
    """
    可视化CQT特征
    :param cqt_data: CQT特征矩阵
    :param sr: 采样率
    :param output_image: 如果提供路径，则保存图片
    """
    plt.figure(figsize=(12, 8))

    librosa.display.specshow(cqt_data, sr=sr, hop_length=512, x_axis='time', y_axis='cqt_note', cmap='turbo')
    plt.title("CQT Features")
    plt.xlabel("Time(s)")
    plt.ylabel("CQT Bins")
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()

    if output_image:
        plt.savefig(output_image)
        print(f"CQT特征图已保存至 {output_image}")
    else:
        plt.show()

In [ ]:
# 示例使用代码
# 假设有音频文件路径
# audio_file = 'path/to/your/audio.wav'

# 加载音频文件
# waveform, sample_rate = torchaudio.load(audio_file)

# 创建CQT变换器
# cqt_transformer = extract_cqt_transformer(sample_rate, n_bins=84)

# 提取CQT特征
# cqt_features = cqt_transformer(waveform)

# 转换为numpy数组便于处理
# cqt_data = cqt_features.squeeze().numpy()

# 保存CQT特征到CSV
# save_cqt_to_csv(cqt_data, 'cqt_features.csv')

# 可视化CQT特征
# plot_cqt_features(cqt_data, sample_rate, 'cqt_features.png')

print("CQT特征提取模块已准备就绪")

In [ ]:
def extract_cqt_with_harmonics(sample_rate, n_bins=84, harmonics=[1, 2, 3]):
    """
    提取CQT特征及其谐波特征
    
    Args:
        sample_rate (int): 采样率
        n_bins (int): CQT频率bins数量
        harmonics (list): 要提取的谐波次数列表
    
    Returns:
        function: 用于提取增强CQT特征的函数
    """
    def extract_features(waveform):
        # 基础CQT变换器
        cqt_transformer = extract_cqt_transformer(sample_rate, n_bins)
        
        # 提取基础CQT特征
        cqt_features = cqt_transformer(waveform)
        cqt_data = cqt_features.squeeze().numpy()
        
        features_list = [cqt_data]
        
        # 提取谐波特征
        for h in harmonics[1:]:  # 跳过基频（h=1）
            # 计算谐波CQT
            harmonic_cqt = librosa.cqt(waveform.squeeze().numpy(), 
                                      sr=sample_rate, 
                                      hop_length=512,
                                      fmin=32.7*h,  # 谐波频率
                                      n_bins=n_bins)
            features_list.append(np.abs(harmonic_cqt))
        
        # 拼接所有特征
        combined_features = np.vstack(features_list)
        
        return combined_features
    
    return extract_features

In [ ]:
def compare_cqt_configurations(waveform, sample_rate):
    """
    比较不同CQT配置的效果
    """
    configurations = [
        {'n_bins': 72, 'bins_per_octave': 12, 'name': 'CQT-72 (6 octaves)'},
        {'n_bins': 84, 'bins_per_octave': 12, 'name': 'CQT-84 (7 octaves)'},
        {'n_bins': 96, 'bins_per_octave': 12, 'name': 'CQT-96 (8 octaves)'}
    ]
    
    fig, axes = plt.subplots(len(configurations), 1, figsize=(12, 4*len(configurations)))
    
    for i, config in enumerate(configurations):
        # 创建CQT变换器
        cqt_transformer = extract_cqt_transformer(sample_rate, config['n_bins'])
        
        # 提取特征
        cqt_features = cqt_transformer(waveform)
        cqt_data = cqt_features.squeeze().numpy()
        
        # 可视化
        ax = axes[i] if len(configurations) > 1 else axes
        librosa.display.specshow(cqt_data, sr=sample_rate, hop_length=512, 
                                x_axis='time', y_axis='cqt_note', ax=ax, cmap='turbo')
        ax.set_title(f"{config['name']} Features")
        ax.set_ylabel('CQT Bins')
        
        print(f"{config['name']} shape: {cqt_data.shape}")
    
    plt.tight_layout()
    plt.show()

In [ ]:
def extract_chromagram_from_cqt(waveform, sample_rate):
    """
    从CQT特征中提取色度图（Chromagram）
    色度图将CQT的频率bins映射到12个半音类别
    """
    # 提取CQT特征
    cqt_transformer = extract_cqt_transformer(sample_rate, n_bins=84)
    cqt_features = cqt_transformer(waveform)
    cqt_data = cqt_features.squeeze().numpy()
    
    # 计算色度图
    chromagram = librosa.feature.chroma_cqt(C=cqt_data, sr=sample_rate)
    
    return chromagram

def plot_chromagram(chromagram, sr, output_image=None):
    """
    可视化色度图
    """
    plt.figure(figsize=(12, 6))
    
    librosa.display.specshow(chromagram, sr=sr, hop_length=512, 
                            x_axis='time', y_axis='chroma', cmap='turbo')
    plt.title('Chromagram from CQT')
    plt.xlabel('Time(s)')
    plt.ylabel('Pitch Class')
    plt.colorbar()
    plt.tight_layout()
    
    if output_image:
        plt.savefig(output_image)
        print(f"色度图已保存至 {output_image}")
    else:
        plt.show()

## CQT特征提取使用说明

### 基本用法
1. 使用 `extract_cqt_transformer()` 创建CQT变换器
2. 使用变换器处理音频波形数据
3. 使用 `save_cqt_to_csv()` 保存特征
4. 使用 `plot_cqt_features()` 可视化特征

### 高级用法
- 使用 `extract_cqt_with_harmonics()` 提取包含谐波特征的增强CQT
- 使用 `compare_cqt_configurations()` 比较不同配置效果
- 使用 `extract_chromagram_from_cqt()` 从CQT中提取色度图

### 参数建议
- **n_bins**: 通常使用84个bins（7个八度音程），音乐分析中较常见
- **bins_per_octave**: 12个bins对应12个半音
- **fmin**: 32.7Hz对应C1音符，适合大多数音乐信号
- **hop_length**: 512提供良好的时间分辨率

### CQT vs 其他时频表示
- **CQT**: 对数频率分辨率，适合音乐信号和谐波分析
- **STFT**: 线性频率分辨率，适合一般音频分析
- **Mel频谱**: 模拟人耳感知，适合语音识别
- **MFCC**: 紧凑表示，适合语音特征提取

### 应用场景
- 音乐信号分析和分类
- 和弦识别和音调分析
- 音乐信息检索
- 船舶音频中的谐波成分分析